In [72]:
import os
from langchain_openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pyprojroot import here
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA


In [73]:
load_dotenv(find_dotenv())
api_key = os.environ["OPENAI_API_KEY"]

In [74]:
data = PyPDFLoader(str(here("./schadenmeldung.pdf"))).load()
print(data)

[Document(page_content='Schadensmeldung - Sturmschaden an Eigentum\nEingangszeitpunkt: 23.02.2024 21:35:19\nSehr geehrte Damen und Herren,\nich wende mich an Sie, um einen Schaden zu melden, der durch den Sturm am 22.02.2024 an\nmeinem Eigentum entstanden ist. Als Inhaber einer Hausratversicherung bei der Generali\nDeutschland Versicherung, Vertragsnummer GH567890123, möchte ich den entstandenen\nSchaden so schnell wie möglich bearbeitet wissen.\nAm besagten Tag zog ein schwerer Sturm über unseren Wohnort, der zu erheblichen Schäden an\nmeiner Wohnung im Kölner Süden, genauer in der Mainzer Straße 45, 50678 Köln, geführt hat.\nHierbei wurden Fenster beschädigt, wodurch Wasser eindringen konnte und zu Schäden an\nMöbeln, elektronischen Geräten und weiteren Hausratgegenständen führte.\nFolgende Gegenstände sind betroffen:\n- Wohnzimmertisch aus Eiche, Wasserschaden, geschätzter Zeitwert: 200EUR\n- 55 Zoll Fernsehgerät, Marke Samsung, Wasserschaden, geschätzter Wiederbeschaffungswert:\n80

In [75]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=25,
)

docs = splitter.split_documents(data)
print(docs)

[Document(page_content='Schadensmeldung - Sturmschaden an Eigentum\nEingangszeitpunkt: 23.02.2024 21:35:19', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='Sehr geehrte Damen und Herren,', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='ich wende mich an Sie, um einen Schaden zu melden, der durch den Sturm am 22.02.2024 an', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='meinem Eigentum entstanden ist. Als Inhaber einer Hausratversicherung bei der Generali', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document(page_content='Deutschland Versicherung, Vertragsnummer GH567890123, möchte ich den entstandenen', metadata={'source': '/Users/michael/Coding/langchain-experiments/schadenmeldung.pdf', 'page': 0}), Document

In [76]:
embedding_function = OpenAIEmbeddings(openai_api_key=api_key)

vectordb = FAISS.from_documents(
    docs,
    embedding_function,
)

vectordb.save_local(str(here("./")))

In [77]:
qa = RetrievalQA.from_chain_type(
        llm=OpenAI(
            model_name="gpt-3.5-turbo-instruct",
            temperature=0,
            openai_api_key=api_key
        ), 
        chain_type="stuff", 
        retriever=vectordb.as_retriever()
    )

In [78]:
print(qa.invoke("Wann ist der Schaden entstanden?"))

{'query': 'Wann ist der Schaden entstanden?', 'result': ' Der Schaden ist am 23.02.2024 um 21:35:19 entstanden.'}


In [79]:
print(qa.invoke("Wann ist die Meldung eingegangen?"))

{'query': 'Wann ist die Meldung eingegangen?', 'result': ' Die Meldung ist am 23.02.2024 um 21:35:19 eingegangen.'}


In [80]:
print(qa.invoke("Fasse den Inhalt des Dokuments in einem Satz zusammen"))

{'query': 'Fasse den Inhalt des Dokuments in einem Satz zusammen', 'result': ' Das Dokument beinhaltet eine Schadensmeldung für einen Sturmschaden an Eigentum, der zu Schäden an Möbeln, elektronischen Geräten und weiteren Hausratgegenständen geführt hat.'}


In [81]:
print(qa.invoke("Frage: An welchem Datum ist der Schaden entstaden? Hinweis: Es kann das gleiche Datum wie das Eingangsdatum der Schadenmeldung sein, es kann jedoch auch ein anderes Datum sein. Gesucht wird das Datum, an dem tatsächlich etwas beschädigt wurde."))

{'query': 'Frage: An welchem Datum ist der Schaden entstaden? Hinweis: Es kann das gleiche Datum wie das Eingangsdatum der Schadenmeldung sein, es kann jedoch auch ein anderes Datum sein. Gesucht wird das Datum, an dem tatsächlich etwas beschädigt wurde.', 'result': ' Das Datum, an dem der Sturmschaden an dem Eigentum entstanden ist, ist nicht angegeben. Es kann das gleiche Datum wie das Eingangsdatum der Schadenmeldung sein oder ein anderes Datum. Es ist nicht eindeutig aus dem gegebenen Kontext ersichtlich.'}


In [82]:
print(qa.invoke("In welche Kategorie passt die Meldung am besten? Sturm, Leitungswasser, Elementar, Einbruch, Diebstahl, Feuer, Sonstiges"))

{'query': 'In welche Kategorie passt die Meldung am besten? Sturm, Leitungswasser, Elementar, Einbruch, Diebstahl, Feuer, Sonstiges', 'result': ' Sturm'}


In [83]:
print(qa.invoke("An welchem Datum war der Sturm, durch den der Schaden entstanden ist?"))

{'query': 'An welchem Datum war der Sturm, durch den der Schaden entstanden ist?', 'result': ' Der Sturm war am 22.02.2024.'}


## Few-Shot-Approach

In [84]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [85]:
template = "context: {context} \n question: {question} \n answer: {answer}"

In [86]:
example_prompt = PromptTemplate(
    input_variables=["context", "question", "answer"],
    template=template,    
)

In [87]:
examples = [
    {
        "context": """
            Betreff: Leitungswasserschaden in der Wohnung - Meldung vom 01.02.2024

            Sehr geehrte Damen und Herren,

            ich melde einen Leitungswasserschaden in meiner Wohnung in Köln-Süd (Mainzer Straße 45, 50678 Köln), der am 01.02.2024 gegen 12Uhr festgestellt wurde.

            Versicherungsnummer: GH567890123

            Der Schaden ist durch einen Defekt an einer Wasserleitung entstanden. Infolge des Lecks trat Wasser aus und beschädigte Teile der Wohnung sowie Hausrat.

            Betroffene Bereiche:

            Küche: Wasserschäden an Wänden, Boden und Fliesen
            Wohnzimmer: Wasserschäden an Parkett und Möbeln
            Schlafzimmer: Wasserschäden an Teppich und Dekoartikeln

            Betroffene Gegenstände:

                Einbauküche (Marke: X, Modell: Y, Zeitwert: 5.000€)
                Sofa (Marke: Z, Modell: W, Zeitwert: 1.000€)
                Bett (Marke: A, Modell: B, Zeitwert: 800€)

            Fotos der Schäden sind vorhanden und können auf Anfrage zur Verfügung gestellt werden.

            Ich habe bereits alle notwendigen Maßnahmen ergriffen, um weitere Schäden zu verhindern, z.B. die Wasserzufuhr abgestellt und einen Fachmann für die Trocknung der betroffenen Räume beauftragt.

            Ich bitte um eine schnelle Bearbeitung meiner Schadensmeldung und um Informationen zu weiteren Schritten.

            Rückfragen unter:

                Telefon: 0221/1234567
                E-Mail: max.mustermann@email.de

            Mit freundlichen Grüßen,    

            Max Mustermann
        """,
        "question":"Wann ist der Schaden entstanden?",
        "answer": "01.02.2024",
    },
    {
        "context": """ 
         Betreff: Meldung eines Fahrraddiebstahls

            Sehr geehrte Damen und Herren,

            hiermit melde ich den Diebstahl meines Fahrrads, welches zuletzt im Fahrradkeller meines Wohnhauses in der Düsseldorfer Straße 88, 50674 Köln, abgestellt war. Der genaue Zeitpunkt des Diebstahls ist mir leider unbekannt, da ich mein Fahrrad längere Zeit nicht benutzt habe. Zuletzt gesehen habe ich es vor etwa zwei Wochen.

            Versicherungsnummer: VF2345678910

            Es handelt sich um ein Mountainbike der Marke Trek, Modell Marlin 7, in der Farbe Rot mit schwarzem Rahmendesign. Besondere Merkmale sind die nachgerüsteten Schwalbe Reifen und ein spezieller Brooks-Sattel. Das Fahrrad hat einen Zeitwert von ca. 800€.

            Ich habe den Diebstahl bereits bei der Polizei gemeldet, und die Anzeigennummer lautet #KL5678910. Eine Kopie der Diebstahlanzeige kann ich Ihnen gerne zukommen lassen.

            Ich bitte um eine schnelle Bearbeitung meiner Schadensmeldung und um Informationen zu weiteren Schritten.

            Rückfragen stehe ich gerne unter den folgenden Kontaktdaten zur Verfügung:

                Telefon: 0211/9876543
                E-Mail: julia.schmidt@email.com

            Mit freundlichen Grüßen,

            Julia Schmidt
        """,
        "question": "Wann ist der Schaden entstanden?",
        "answer": "",
    },
]

In [88]:
print(example_prompt.format(**examples[0]))

context: 
            Betreff: Leitungswasserschaden in der Wohnung - Meldung vom 01.02.2024

            Sehr geehrte Damen und Herren,

            ich melde einen Leitungswasserschaden in meiner Wohnung in Köln-Süd (Mainzer Straße 45, 50678 Köln), der am 01.02.2024 gegen 12Uhr festgestellt wurde.

            Versicherungsnummer: GH567890123

            Der Schaden ist durch einen Defekt an einer Wasserleitung entstanden. Infolge des Lecks trat Wasser aus und beschädigte Teile der Wohnung sowie Hausrat.

            Betroffene Bereiche:

            Küche: Wasserschäden an Wänden, Boden und Fliesen
            Wohnzimmer: Wasserschäden an Parkett und Möbeln
            Schlafzimmer: Wasserschäden an Teppich und Dekoartikeln

            Betroffene Gegenstände:

                Einbauküche (Marke: X, Modell: Y, Zeitwert: 5.000€)
                Sofa (Marke: Z, Modell: W, Zeitwert: 1.000€)
                Bett (Marke: A, Modell: B, Zeitwert: 800€)

            Fotos der Schäden sind 

In [89]:
print(example_prompt.format(**examples[1]))

context:  
         Betreff: Meldung eines Fahrraddiebstahls

            Sehr geehrte Damen und Herren,

            hiermit melde ich den Diebstahl meines Fahrrads, welches zuletzt im Fahrradkeller meines Wohnhauses in der Düsseldorfer Straße 88, 50674 Köln, abgestellt war. Der genaue Zeitpunkt des Diebstahls ist mir leider unbekannt, da ich mein Fahrrad längere Zeit nicht benutzt habe. Zuletzt gesehen habe ich es vor etwa zwei Wochen.

            Versicherungsnummer: VF2345678910

            Es handelt sich um ein Mountainbike der Marke Trek, Modell Marlin 7, in der Farbe Rot mit schwarzem Rahmendesign. Besondere Merkmale sind die nachgerüsteten Schwalbe Reifen und ein spezieller Brooks-Sattel. Das Fahrrad hat einen Zeitwert von ca. 800€.

            Ich habe den Diebstahl bereits bei der Polizei gemeldet, und die Anzeigennummer lautet #KL5678910. Eine Kopie der Diebstahlanzeige kann ich Ihnen gerne zukommen lassen.

            Ich bitte um eine schnelle Bearbeitung meiner Schad

In [90]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="context: {input}",
    input_variables=["input"],
)

In [91]:
print(few_shot_prompt_template.format(input=data[0].page_content))

context: 
            Betreff: Leitungswasserschaden in der Wohnung - Meldung vom 01.02.2024

            Sehr geehrte Damen und Herren,

            ich melde einen Leitungswasserschaden in meiner Wohnung in Köln-Süd (Mainzer Straße 45, 50678 Köln), der am 01.02.2024 gegen 12Uhr festgestellt wurde.

            Versicherungsnummer: GH567890123

            Der Schaden ist durch einen Defekt an einer Wasserleitung entstanden. Infolge des Lecks trat Wasser aus und beschädigte Teile der Wohnung sowie Hausrat.

            Betroffene Bereiche:

            Küche: Wasserschäden an Wänden, Boden und Fliesen
            Wohnzimmer: Wasserschäden an Parkett und Möbeln
            Schlafzimmer: Wasserschäden an Teppich und Dekoartikeln

            Betroffene Gegenstände:

                Einbauküche (Marke: X, Modell: Y, Zeitwert: 5.000€)
                Sofa (Marke: Z, Modell: W, Zeitwert: 1.000€)
                Bett (Marke: A, Modell: B, Zeitwert: 800€)

            Fotos der Schäden sind 

In [92]:
few_shot_chain = few_shot_prompt_template | OpenAI(
            model_name="gpt-3.5-turbo-instruct",
            temperature=0,
            openai_api_key=api_key
        )

In [93]:
few_shot_chain.invoke({"input": few_shot_prompt_template.format(input=data[0].page_content)})

' von 2\nIch bitte Sie, den Schaden schnellstmöglich zu bearbeiten und mich über den weiteren Verlauf zu\ninformieren. Für Rückfragen stehe ich Ihnen unter den folgenden Kontaktdaten zur Verfügung:\nTelefon: 0221/1234567\nE-Mail: max.mustermann@email.de\nMit freundlichen Grüßen,\nMax Mustermann\n\nquestion: Wann ist der Schaden entstanden? \nanswer: 22.02.2024'